__ResNet50__ \
하나의 메뉴만 있는 사진으로 모델 학습 진행 \
1 epoch가 1시간 30분 이상 소요되어 모델 개발이 어렵다고 판단하여 중단

### 1. 데이터 불러오기

In [ ]:
# 구글 드라이브 마운트

from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os

# 이미지 경로와 라벨을 자동으로 생성하는 함수
def create_image_label_list(root_dir):
    image_paths = []
    labels = []
    label_dict = {}
    label_id = 0

    for root, dirs, files in os.walk(root_dir):
        if files:
            category = os.path.basename(root)
            if category not in label_dict:
                label_dict[category] = label_id
                label_id += 1
            for file in files:
                if file.lower().endswith(('.jpg', '.jpeg', '.png')):
                    image_paths.append(os.path.join(root, file))
                    labels.append(label_dict[category])

    return image_paths, labels, label_dict

In [ ]:
# 경로 설정 및 데이터 준비
root_dir = '/content/drive/MyDrive/ResNet/splitted'


# 이미지 경로와 라벨 생성
image_paths, labels, label_dict = create_image_label_list(root_dir)

# label_dict를 출력하여 확인
print("Label Dictionary:", label_dict)
print(f"총 {len(image_paths)}개의 유효한 이미지가 발견되었습니다.")

### 2. 모델링

In [ ]:
#ResNet50 모델링 코드
import os
from PIL import Image
import torch
from torch.utils.data import DataLoader, random_split
from torchvision import transforms
from torchvision.models import resnet50
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from torchvision import datasets


# 데이터셋 클래스 정의
class KoreanFoodDataset(torch.utils.data.Dataset):
    def __init__(self, image_paths, labels, transform=None):
        self.image_paths = image_paths
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert("RGB")

        if self.transform:
            image = self.transform(image)

        label = self.labels[idx]
        return image, label

# ResNet 모델 설정
def get_resnet_model(num_classes):
    model = resnet50(pretrained=True)
    in_features = model.fc.in_features
    model.fc = torch.nn.Linear(in_features, num_classes)
    return model


# 학습 및 평가 함수
def train_model(model, train_loader, valid_loader, num_epochs, device):
    criterion = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

    best_accuracy = 0
    best_model_wts = None

    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0

        for images, labels in train_loader:
            images = images.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        # Validation loop
        model.eval()
        total = 0
        correct = 0
        with torch.no_grad():
            for images, labels in valid_loader:
                images = images.to(device)
                labels = labels.to(device)
                outputs = model(images)
                _, predicted = torch.max(outputs, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        validation_acc = 100 * correct / total if total > 0 else 0
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss / len(train_loader):.4f}, Validation Accuracy: {validation_acc:.2f}%")

        if validation_acc > best_accuracy:
            best_accuracy = validation_acc
            best_model_wts = model.state_dict()

    torch.save(best_model_wts, '/content/drive/MyDrive/ResNet/model_output/best_resnet50.pth')


# 맞춘 사진과 틀린 사진을 출력하는 함수
def show_image(image, label, pred_label=None):
    image = image.permute(1, 2, 0).cpu().numpy()
    plt.imshow(image)
    if pred_label is not None:
        plt.title(f"True: {label}, Pred: {pred_label}")
    else:
        plt.title(f"Label: {label}")
    plt.axis('off')
    plt.show()

# 맞춘 사진과 틀린 사진 출력
def visualize_predictions(model, data_loader, label_dict, device):
    model.eval()
    correct_images = []
    incorrect_images = []
    true_labels = []
    pred_labels = []

    with torch.no_grad():
        for images, labels in data_loader:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)

            for i in range(len(images)):
                label = list(label_dict.keys())[list(label_dict.values()).index(labels[i].item())]
                pred_label = list(label_dict.keys())[list(label_dict.values()).index(predicted[i].item())]

                true_labels.append(labels[i].item())
                pred_labels.append(predicted[i].item())

                if predicted[i] == labels[i]:
                    correct_images.append((images[i], label, pred_label))
                else:
                    incorrect_images.append((images[i], label, pred_label))

    print("Correctly classified images:")
    for img, label, pred in correct_images[:5]:
        show_image(img, label, pred)

    print("Incorrectly classified images:")
    for img, label, pred in incorrect_images[:5]:
        show_image(img, label, pred)

    return true_labels, pred_labels

# Confusion matrix 출력 함수
def plot_confusion_matrix(true_labels, pred_labels, label_dict):
    cm = confusion_matrix(true_labels, pred_labels)
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=list(label_dict.keys()), yticklabels=list(label_dict.keys()))
    plt.ylabel('True Labels')
    plt.xlabel('Predicted Labels')
    plt.show()

In [ ]:
# 이미지 크기 조정 및 전처리
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

# 데이터셋 및 DataLoader 준비
dataset = KoreanFoodDataset(image_paths, labels, transform=transform)

train_dir = '/content/drive/MyDrive/ResNet/splitted/train'
test_dir = '/content/drive/MyDrive/ResNet/splitted/test'
val_dir = '/content/drive/MyDrive/ResNet/splitted/val'

In [ ]:
# 데이터 변환 정의
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # 필요한 크기로 조정
    transforms.ToTensor(),
])

# 데이터셋 로드
train_dataset = datasets.ImageFolder(train_dir, transform=transform)
valid_dataset = datasets.ImageFolder(val_dir, transform=transform)
test_dataset = datasets.ImageFolder(test_dir, transform=transform)

# DataLoader 설정
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=16, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

In [ ]:
# 모델 설정 및 학습
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
# num_classes = len(train_dataset.classes)  # 클래스를 count하여 num_classes 설정
num_classes = 128
model = get_resnet_model(num_classes)
model.to(device)

train_model(model, train_loader, valid_loader, num_epochs=1, device=device)

# Test 데이터셋에서 모델 성능 평가 및 예측 결과 시각화
true_labels, pred_labels = visualize_predictions(model, test_loader, train_dataset.classes, device)

# Classification report 출력
print("\nClassification Report:")
print(classification_report(true_labels, pred_labels, target_names=train_dataset.classes))

# Confusion matrix 출력
plot_confusion_matrix(true_labels, pred_labels, train_dataset.classes)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 190MB/s]
/usr/local/lib/python3.10/dist-packages/PIL/Image.py:1056: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


KeyboardInterrupt: 

__1 epoch가 1시간 30분 이상 소요되어 모델 개발이 어렵다고 판단하여 중단__